In [278]:
import numpy as np
import pandas as pd 

In [279]:
df_train = pd.read_csv("../input/credit-dset/train.csv")

/tmp/ipykernel_30/403186247.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("../input/credit-dset/train.csv")


In [280]:
# string -> no of months
def convert_to_months(age_str):
    if pd.isna(age_str):
        return np.nan
    else : 
        parts = age_str.split(' and ')
        years = int(parts[0].split()[0])
        months = int(parts[1].split()[0])
        total_months = (years * 12) + months
        return total_months

In [281]:
loan_type_col = df_train['Loan_Type']
# dropping columns
df_train = df_train.drop(['Name','Loan_Type','ID'], axis=1)
# base salary -> number
df_train['Base_Salary_PerMonth'] = pd.to_numeric(df_train['Base_Salary_PerMonth'],downcast = 'float',errors = 'coerce')
# Total delayed payments (removing underscores if present) -> number
df_train['Total_Delayed_Payments'] = df_train['Total_Delayed_Payments'].str.replace(r'[^-0-9]', '', regex=True)
df_train['Total_Delayed_Payments'] = pd.to_numeric(df_train['Total_Delayed_Payments'],downcast = 'float',errors = 'coerce')
# credit history age -> number (to no of months)
df_train['Credit_History_Age'] = df_train['Credit_History_Age'].apply(convert_to_months)
df_train['Credit_History_Age'] = pd.to_numeric(df_train['Credit_History_Age'],downcast = 'float',errors = 'coerce')
#age -> number
df_train['Age'] = df_train['Age'].str.replace(r'[^-0-9]', '', regex=True)
df_train['Age'] = pd.to_numeric(df_train['Age'],downcast = 'integer',errors = 'coerce')
#Income_annual -> number
df_train['Income_Annual'] = df_train['Income_Annual'].str.replace(r'[^-.0-9]', '', regex=True)
df_train['Income_Annual'] = pd.to_numeric(df_train['Income_Annual'],downcast = 'float',errors = 'coerce')
#Total_Current_Loans -> number
df_train['Total_Current_Loans'] = df_train['Total_Current_Loans'].str.replace(r'[^-0-9]', '', regex=True)
df_train['Total_Current_Loans'] = pd.to_numeric(df_train['Total_Current_Loans'],downcast = 'integer',errors = 'coerce')
#Current_Debt_Outstanding -> number
df_train['Current_Debt_Outstanding'] = df_train['Current_Debt_Outstanding'].str.replace(r'[^-.0-9]', '', regex=True)
df_train['Current_Debt_Outstanding'] = pd.to_numeric(df_train['Current_Debt_Outstanding'],downcast = 'float',errors = 'coerce')
#Credit_Limit -> number
df_train['Credit_Limit'] = pd.to_numeric(df_train['Credit_Limit'],downcast = 'float',errors = 'coerce')
#Monthly_Balance -> number
df_train['Monthly_Balance'] = pd.to_numeric(df_train['Monthly_Balance'],downcast = 'float',errors = 'coerce')

In [282]:
print(df_train.info(),end = "\n\n")
col = "Credit_Limit" #Monthly_Balance
print(df_train[pd.to_numeric(df_train[col],downcast = 'float', errors="coerce").isna()][col].value_counts())
#print(df_train[col].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               80000 non-null  object 
 1   Month                     80000 non-null  object 
 2   Age                       80000 non-null  int16  
 3   Number                    80000 non-null  object 
 4   Profession                80000 non-null  object 
 5   Income_Annual             80000 non-null  float64
 6   Base_Salary_PerMonth      67968 non-null  float32
 7   Total_Bank_Accounts       80000 non-null  int64  
 8   Total_Credit_Cards        80000 non-null  int64  
 9   Rate_Of_Interest          80000 non-null  int64  
 10  Total_Current_Loans       80000 non-null  int16  
 11  Delay_from_due_date       80000 non-null  int64  
 12  Total_Delayed_Payments    74405 non-null  float32
 13  Credit_Limit              78309 non-null  float32
 14  Total_

In [283]:
df_train.drop_duplicates(inplace=True)
print(df_train.isna().sum().to_string())
print(df_train.shape)

Customer_ID                     0
Month                           0
Age                             0
Number                          0
Profession                      0
Income_Annual                   0
Base_Salary_PerMonth        12032
Total_Bank_Accounts             0
Total_Credit_Cards              0
Rate_Of_Interest                0
Total_Current_Loans             0
Delay_from_due_date             0
Total_Delayed_Payments       5595
Credit_Limit                 1691
Total_Credit_Enquiries       1549
Credit_Mix                      0
Current_Debt_Outstanding        0
Ratio_Credit_Utilization        0
Credit_History_Age           7240
Payment_of_Min_Amount           0
Per_Month_EMI                   0
Monthly_Investment           3605
Payment_Behaviour               0
Monthly_Balance               957
Credit_Score                    0
(80000, 25)


In [284]:
df_train["Credit_Limit"] = df_train.groupby("Customer_ID")["Credit_Limit"].transform(
        lambda x: x.fillna(method="ffill").fillna(method="bfill")
)
df_train['Credit_Limit'].isna().sum()

/tmp/ipykernel_30/2762047474.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.fillna(method="ffill").fillna(method="bfill")


0

In [285]:
unknown_min_pay_repl = df_train['Payment_of_Min_Amount'].mode()[0]
df_train['Payment_of_Min_Amount'] = df_train.groupby("Customer_ID")['Payment_of_Min_Amount'].transform(
    lambda x: x.where(x != "NM", x[x != "NM"].mode().get(0, unknown_min_pay_repl))
)
df_train['Payment_of_Min_Amount'].value_counts()

Payment_of_Min_Amount
Yes    47543
No     32457
Name: count, dtype: int64

In [286]:
unknown_mix_repl = df_train['Credit_Mix'].mode()[0]
df_train['Credit_Mix'] = df_train.groupby("Customer_ID")['Credit_Mix'].transform(
    lambda x: x.where(x != "_", x[x != "_"].mode().get(0, unknown_mix_repl))
)
df_train['Credit_Mix'].value_counts()

Credit_Mix
Standard    36699
Good        24275
Bad         19026
Name: count, dtype: int64

In [287]:
unknown_prof_repl = 'Lawyer'
df_train['Profession'] = df_train.groupby("Customer_ID")['Profession'].transform(
    lambda x: x.where(x != "_______", x[x != "_______"].mode().get(0, unknown_prof_repl))
)
df_train['Profession'].value_counts()

Profession
Lawyer           5692
Engineer         5497
Mechanic         5493
Architect        5479
Scientist        5394
Accountant       5389
Media_Manager    5363
Teacher          5344
Developer        5343
Journalist       5281
Entrepreneur     5235
Doctor           5214
Manager          5172
Musician         5065
Writer           5039
Name: count, dtype: int64

In [288]:
unknown_number_repl = '000-00-0000'
df_train['Number'] = df_train.groupby("Customer_ID")['Number'].transform(
    lambda x: x.where(x != "#F%$D@*&8", x[x != "#F%$D@*&8"].mode().get(0, unknown_number_repl))
)
df_train['Number'].value_counts()

Number
865-33-5347    8
808-71-5249    8
676-82-0055    8
430-15-0729    8
309-02-3479    8
              ..
702-39-1597    2
346-46-1263    2
837-45-0195    2
761-53-5627    2
789-15-7049    2
Name: count, Length: 12500, dtype: int64

In [289]:
unknown_behavior_repl = df_train['Payment_Behaviour'].mode()[0]
df_train['Payment_Behaviour'] = df_train.groupby("Customer_ID")['Payment_Behaviour'].transform(
    lambda x: x.where(x != "!@9#%8", x[x != "!@9#%8"].mode().get(0, unknown_behavior_repl))
)
df_train['Payment_Behaviour'].value_counts()

Payment_Behaviour
Low_spent_Small_value_payments      22144
High_spent_Medium_value_payments    15452
High_spent_Large_value_payments     12375
Low_spent_Medium_value_payments     11724
High_spent_Small_value_payments      9564
Low_spent_Large_value_payments       8741
Name: count, dtype: int64

In [290]:
null_percentages=(df_train.isna().sum()/df_train.shape[0])*100
null_cols = null_percentages.loc[null_percentages > 0]
null_cols

Base_Salary_PerMonth      15.04000
Total_Delayed_Payments     6.99375
Total_Credit_Enquiries     1.93625
Credit_History_Age         9.05000
Monthly_Investment         4.50625
Monthly_Balance            1.19625
dtype: float64

In [291]:
rows_to_drop = null_cols.loc[null_cols < 5]
df_train.dropna(subset = rows_to_drop.keys(),inplace=True,how='any',axis=0)
print(df_train.isna().sum().to_string())
print(df_train.shape)

Customer_ID                     0
Month                           0
Age                             0
Number                          0
Profession                      0
Income_Annual                   0
Base_Salary_PerMonth        11109
Total_Bank_Accounts             0
Total_Credit_Cards              0
Rate_Of_Interest                0
Total_Current_Loans             0
Delay_from_due_date             0
Total_Delayed_Payments       5184
Credit_Limit                    0
Total_Credit_Enquiries          0
Credit_Mix                      0
Current_Debt_Outstanding        0
Ratio_Credit_Utilization        0
Credit_History_Age           6693
Payment_of_Min_Amount           0
Per_Month_EMI                   0
Monthly_Investment              0
Payment_Behaviour               0
Monthly_Balance                 0
Credit_Score                    0
(74021, 25)


In [292]:
columns_to_drop = null_cols.loc[null_cols > 40]
df_train.drop(columns = columns_to_drop.keys(),inplace = True)
df_train.drop_duplicates(inplace=True)
print(df_train.shape)

(74021, 25)


In [293]:
null_percentages=(df_train.isna().sum()/df_train.shape[0])*100
null_cols = null_percentages.loc[null_percentages > 0]
print(null_cols,end = "\n\n")
col_impute = null_cols.loc[(null_cols >= 5) & (null_cols < 40)]
for column in col_impute.keys():
    central_tend = df_train[column].mean()
    df_train[column] = df_train[column].fillna(central_tend)

Base_Salary_PerMonth      15.007903
Total_Delayed_Payments     7.003418
Credit_History_Age         9.042029
dtype: float64



In [294]:
df_train['Monthly_Investment'] = df_train['Monthly_Investment'].str.replace(r'[^-.0-9]', '', regex=True)
df_train['Monthly_Investment'] = pd.to_numeric(df_train['Monthly_Investment'],downcast = 'float',errors = 'coerce')
df_train = df_train.drop(['Customer_ID'], axis=1)
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 74021 entries, 0 to 79999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     74021 non-null  object 
 1   Age                       74021 non-null  int16  
 2   Number                    74021 non-null  object 
 3   Profession                74021 non-null  object 
 4   Income_Annual             74021 non-null  float64
 5   Base_Salary_PerMonth      74021 non-null  float32
 6   Total_Bank_Accounts       74021 non-null  int64  
 7   Total_Credit_Cards        74021 non-null  int64  
 8   Rate_Of_Interest          74021 non-null  int64  
 9   Total_Current_Loans       74021 non-null  int16  
 10  Delay_from_due_date       74021 non-null  int64  
 11  Total_Delayed_Payments    74021 non-null  float32
 12  Credit_Limit              74021 non-null  float32
 13  Total_Credit_Enquiries    74021 non-null  float64
 14  Credit_Mix 

In [295]:
df_train.to_csv('clean_trained.csv',index = False)

In [296]:
df_test = pd.read_csv('../input/credit-dset/test.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        20000 non-null  object 
 1   Customer_ID               20000 non-null  object 
 2   Month                     20000 non-null  object 
 3   Name                      18044 non-null  object 
 4   Age                       20000 non-null  object 
 5   Number                    20000 non-null  object 
 6   Profession                20000 non-null  object 
 7   Income_Annual             20000 non-null  object 
 8   Base_Salary_PerMonth      17030 non-null  float64
 9   Total_Bank_Accounts       20000 non-null  int64  
 10  Total_Credit_Cards        20000 non-null  int64  
 11  Rate_Of_Interest          20000 non-null  int64  
 12  Total_Current_Loans       20000 non-null  object 
 13  Loan_Type                 17749 non-null  object 
 14  Delay_

In [297]:
# loan_type_col = df_test['Loan_Type']
# dropping columns
df_test = df_test.drop(['Name','Loan_Type'], axis=1)
# base salary -> number
df_test['Base_Salary_PerMonth'] = pd.to_numeric(df_test['Base_Salary_PerMonth'],downcast = 'float',errors = 'coerce')
# Total delayed payments (removing underscores if present) -> number
df_test['Total_Delayed_Payments'] = df_test['Total_Delayed_Payments'].str.replace(r'[^-0-9]', '', regex=True)
df_test['Total_Delayed_Payments'] = pd.to_numeric(df_test['Total_Delayed_Payments'],downcast = 'float',errors = 'coerce')
# credit history age -> number (to no of months)
df_test['Credit_History_Age'] = df_test['Credit_History_Age'].apply(convert_to_months)
df_test['Credit_History_Age'] = pd.to_numeric(df_test['Credit_History_Age'],downcast = 'float',errors = 'coerce')
#age -> number
df_test['Age'] = df_test['Age'].str.replace(r'[^-0-9]', '', regex=True)
df_test['Age'] = pd.to_numeric(df_test['Age'],downcast = 'integer',errors = 'coerce')
#Income_annual -> number
df_test['Income_Annual'] = df_test['Income_Annual'].str.replace(r'[^-.0-9]', '', regex=True)
df_test['Income_Annual'] = pd.to_numeric(df_test['Income_Annual'],downcast = 'float',errors = 'coerce')
#Total_Current_Loans -> number
df_test['Total_Current_Loans'] = df_test['Total_Current_Loans'].str.replace(r'[^-0-9]', '', regex=True)
df_test['Total_Current_Loans'] = pd.to_numeric(df_test['Total_Current_Loans'],downcast = 'integer',errors = 'coerce')
#Current_Debt_Outstanding -> number
df_test['Current_Debt_Outstanding'] = df_test['Current_Debt_Outstanding'].str.replace(r'[^-.0-9]', '', regex=True)
df_test['Current_Debt_Outstanding'] = pd.to_numeric(df_test['Current_Debt_Outstanding'],downcast = 'float',errors = 'coerce')
#Credit_Limit -> number
df_test['Credit_Limit'] = pd.to_numeric(df_test['Credit_Limit'],downcast = 'float',errors = 'coerce')
#Monthly_Balance -> number
df_test['Monthly_Balance'] = pd.to_numeric(df_test['Monthly_Balance'],downcast = 'float',errors = 'coerce')
#Monthlu_invetment
df_test['Monthly_Investment'] = df_test['Monthly_Investment'].str.replace(r'[^-.0-9]', '', regex=True)
df_test['Monthly_Investment'] = pd.to_numeric(df_test['Monthly_Investment'],downcast = 'float',errors = 'coerce')

In [298]:
print(df_test.isna().sum().to_string(),end = "\n\n")
print(df_test.info())

ID                             0
Customer_ID                    0
Month                          0
Age                            0
Number                         0
Profession                     0
Income_Annual                  0
Base_Salary_PerMonth        2970
Total_Bank_Accounts            0
Total_Credit_Cards             0
Rate_Of_Interest               0
Total_Current_Loans            0
Delay_from_due_date            0
Total_Delayed_Payments      1407
Credit_Limit                 400
Total_Credit_Enquiries       416
Credit_Mix                     0
Current_Debt_Outstanding       0
Ratio_Credit_Utilization       0
Credit_History_Age          1790
Payment_of_Min_Amount          0
Per_Month_EMI                  0
Monthly_Investment           874
Payment_Behaviour              0
Monthly_Balance              252

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  -----

In [299]:
col = "Payment_Behaviour" #Monthly_Balance
#print(df_test[pd.to_numeric(df_test[col],downcast = 'float', errors="coerce").isna()][col].value_counts())
print(df_test[col].value_counts())

Payment_Behaviour
Low_spent_Small_value_payments      5043
High_spent_Medium_value_payments    3483
High_spent_Large_value_payments     2790
Low_spent_Medium_value_payments     2760
High_spent_Small_value_payments     2306
Low_spent_Large_value_payments      2116
!@9#%8                              1502
Name: count, dtype: int64


In [300]:
# unknown_base_repl_test = df_test['Base_Salary_PerMonth'].mean()
base_sal_mean_test = df_test['Base_Salary_PerMonth'].mean()
df_test['Base_Salary_PerMonth'] = df_test.groupby("Customer_ID")['Base_Salary_PerMonth'].transform(
    lambda x: x.fillna(x.mean() if not x.mean() != x.mean() else base_sal_mean_test)#.get(0, unknown_min_pay_repl)
)
df_test['Base_Salary_PerMonth'].isna().sum()

0

In [301]:
base_sal_mean_test = df_test['Total_Delayed_Payments'].mode()[0]
df_test['Total_Delayed_Payments'] = df_test.groupby("Customer_ID")['Total_Delayed_Payments'].transform(
    lambda x: x.fillna(x[x.notna()].mode().get(0, base_sal_mean_test))
)
df_test['Total_Delayed_Payments'].isna().sum()

0

In [302]:
df_test["Credit_Limit"] = df_test.groupby("Customer_ID")["Credit_Limit"].transform(
        lambda x: x.fillna(method="ffill").fillna(method="bfill")
)
df_train['Credit_Limit'].isna().sum()

/tmp/ipykernel_30/2019341072.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.fillna(method="ffill").fillna(method="bfill")


0

In [303]:
base_sal_mean_test = df_test['Total_Credit_Enquiries'].mean()
df_test['Total_Credit_Enquiries'] = df_test.groupby("Customer_ID")['Total_Credit_Enquiries'].transform(
    lambda x: x.fillna(x[x.notna()].mode().get(0, base_sal_mean_test))
)
df_test['Total_Credit_Enquiries'].isna().sum()

0

In [304]:
base_sal_mean_test = df_test['Credit_History_Age'].mean()
df_test['Credit_History_Age'] = df_test.groupby("Customer_ID")['Credit_History_Age'].transform(
    lambda x: x.fillna(x[x.notna()].mode().get(0, base_sal_mean_test))
)
df_test['Credit_History_Age'].isna().sum()

0

In [305]:
base_sal_mean_test = df_test['Monthly_Investment'].mean()
df_test['Monthly_Investment'] = df_test.groupby("Customer_ID")['Monthly_Investment'].transform(
    lambda x: x.fillna(x[x.notna()].mode().get(0, base_sal_mean_test))
)
df_test['Monthly_Investment'].isna().sum()

0

In [306]:
base_sal_mean_test = df_test['Monthly_Balance'].mean()
df_test['Monthly_Balance'] = df_test.groupby("Customer_ID")['Monthly_Balance'].transform(
    lambda x: x.fillna(x[x.notna()].mode().get(0, base_sal_mean_test))
)
df_test['Monthly_Balance'].isna().sum()

0

In [307]:
unknown_prof_repl = 'Lawyer'
df_test['Profession'] = df_test.groupby("Customer_ID")['Profession'].transform(
    lambda x: x.where(x != "_______", x[x != "_______"].mode().get(0, unknown_prof_repl))
)
df_train['Profession'].value_counts()

Profession
Lawyer           5316
Mechanic         5088
Engineer         5087
Architect        5055
Media_Manager    4982
Teacher          4973
Scientist        4962
Accountant       4957
Developer        4947
Journalist       4868
Entrepreneur     4868
Doctor           4815
Manager          4797
Writer           4658
Musician         4648
Name: count, dtype: int64

In [308]:
unknown_number_repl = '000-00-0000'
df_test['Number'] = df_test.groupby("Customer_ID")['Number'].transform(
    lambda x: x.where(x != "#F%$D@*&8", x[x != "#F%$D@*&8"].mode().get(0, unknown_number_repl))
)
df_train['Number'].value_counts()

Number
938-66-9299    8
534-32-1760    8
232-16-0697    8
766-27-1126    8
199-63-4034    8
              ..
202-26-2578    1
578-81-4278    1
967-55-6862    1
424-23-9837    1
214-02-8382    1
Name: count, Length: 12500, dtype: int64

In [309]:
unknown_min_pay_repl = df_test['Payment_of_Min_Amount'].mode()[0]
df_test['Payment_of_Min_Amount'] = df_test.groupby("Customer_ID")['Payment_of_Min_Amount'].transform(
    lambda x: x.where(x != "NM", x[x != "NM"].mode().get(0, unknown_min_pay_repl))
)
df_test['Payment_of_Min_Amount'].value_counts()

Payment_of_Min_Amount
Yes    12153
No      7847
Name: count, dtype: int64

In [310]:
unknown_behavior_repl = df_test['Payment_Behaviour'].mode()[0]
df_test['Payment_Behaviour'] = df_test.groupby("Customer_ID")['Payment_Behaviour'].transform(
    lambda x: x.where(x != "!@9#%8", x[x != "!@9#%8"].mode().get(0, unknown_behavior_repl))
)
df_test['Payment_Behaviour'].value_counts()

Payment_Behaviour
Low_spent_Small_value_payments      5603
High_spent_Medium_value_payments    3741
High_spent_Large_value_payments     3062
Low_spent_Medium_value_payments     2892
High_spent_Small_value_payments     2465
Low_spent_Large_value_payments      2237
Name: count, dtype: int64

In [311]:
unknown_mix_repl = df_test['Credit_Mix'].mode()[0]
df_test['Credit_Mix'] = df_test.groupby("Customer_ID")['Credit_Mix'].transform(
    lambda x: x.where(x != "_", x[x != "_"].mode().get(0, unknown_mix_repl))
)
df_test['Credit_Mix'].value_counts()

Credit_Mix
Standard    9833
Good        5739
Bad         4428
Name: count, dtype: int64

In [312]:
df_test = df_test.drop(['Customer_ID'], axis=1)
df_test.info()
df_test.to_csv('test_cleaned.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        20000 non-null  object 
 1   Month                     20000 non-null  object 
 2   Age                       20000 non-null  int16  
 3   Number                    20000 non-null  object 
 4   Profession                20000 non-null  object 
 5   Income_Annual             20000 non-null  float64
 6   Base_Salary_PerMonth      20000 non-null  float32
 7   Total_Bank_Accounts       20000 non-null  int64  
 8   Total_Credit_Cards        20000 non-null  int64  
 9   Rate_Of_Interest          20000 non-null  int64  
 10  Total_Current_Loans       20000 non-null  int16  
 11  Delay_from_due_date       20000 non-null  int64  
 12  Total_Delayed_Payments    20000 non-null  float32
 13  Credit_Limit              19926 non-null  float32
 14  Total_